<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Essentially" data-toc-modified-id="Essentially-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Essentially</a></span><ul class="toc-item"><li><span><a href="#What's-happening?" data-toc-modified-id="What's-happening?-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>What's happening?</a></span></li><li><span><a href="#weave" data-toc-modified-id="weave-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>weave</a></span></li><li><span><a href="#unweave" data-toc-modified-id="unweave-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>unweave</a></span></li><li><span><a href="#inspecting-the-parts-of-the-woven" data-toc-modified-id="inspecting-the-parts-of-the-woven-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>inspecting the parts of the woven</a></span></li><li><span><a href="#a-bit-more-control-on-how-you-get-your-woven-parts" data-toc-modified-id="a-bit-more-control-on-how-you-get-your-woven-parts-1.5"><span class="toc-item-num">1.5&nbsp;&nbsp;</span>a bit more control on how you get your woven parts</a></span></li></ul></li></ul></div>

In [118]:
from owner import int_to_bytes, bytes_to_int

# int_to_bytes(bytes_to_int())

bytes_to_int(int_to_bytes(123456)) == 123456
int_to_bytes(7, n_bytes=3)
int_to_bytes(255, n_bytes=3)
int_to_bytes(256, n_bytes=3)
int_to_bytes(256 + 7, n_bytes=3)

b'\x00\x01\x07'

# Essentially

In [98]:
from owner import weave, unweave

assert unweave(weave(b'bob and alice')) == b'bob and alice'

## What's happening?

Well, here's what's happening by default.

First, a weaver is made...

In [97]:
from owner import HeadWeaver

weaver = HeadWeaver()

## weave

When we weave, we get the same original content, but with a bunch of "other junk"...

In [91]:
content = b'--this is where content goes--'
extra_info = b'**optional free-from "extra info goes here: It is meant for ownership and dating info.**'
w = weaver.weave(content, extra_info)
w

b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8\xe8gU\xcc\xc1\n)\xd1\xb0\x1dd\x07\xa4\x90\xbd\xfe\x0f7\xe9\xb5\xcd\xf8\xd7[z\xc5\xeao\xac\x03|8**optional free-from "extra info goes here: It is meant for ownership and dating info.**--this is where content goes--'

## unweave

No fret though, we can unjunk the weave.

In [93]:
unwoven_content = weaver.unweave(w)
assert unwoven_content == content  # see that the unwoven content is the same as the original content
unwoven_content

b'--this is where content goes--'

What does weaving and unweaving do for you?

Well, that's up to you. It's just a means to add some information to some content, within the bytes of the content itself, and be able to still access the original content. 

What you put in the `extra_info` depends on you and your use case. 
The use case we have in mind here is: I want to prove that some content is mine by posting a hash of it on a block-chain. 
But if I just hash the content, only the content is represented, not any other information (such as my name etc.). 
Simple solution to that: Just prepend my name (or any other information) to the raw content.
That's what `extra_info` is.
Ah... but then I need to know where the original content starts. 
That's what `offset` is.
And then we added a `content_hash` to add extra error-correcting abilities. 

Let's now have a look at how the woven bytes are parsed.

## inspecting the parts of the woven

In [66]:
parts = weaver.unweave_parts(w)
parts

UnWovenBaseHeaderWeave(offset=b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xb8', content_hash=b'\xe8gU\xcc\xc1\n)\xd1\xb0\x1dd\x07\xa4\x90\xbd\xfe\x0f7\xe9\xb5\xcd\xf8\xd7[z\xc5\xeao\xac\x03|8', extra_info=b'**optional free-from "extra info goes here: It is meant for ownership and dating info.**', content=b'--this is where content goes--')

In [67]:
parts._fields

('offset', 'content_hash', 'extra_info', 'content')

As you see in the print out above, the parts are all given in raw bytes format. 

If you need them to be interpreted you can do so like this:

In [68]:
parts = weaver.unweave_parts(w, interpret_bytes=True)
parts

UnWovenBaseHeaderWeave(offset=184, content_hash='e86755ccc10a29d1b01d6407a490bdfe0f37e9b5cdf8d75b7ac5ea6fac037c38', extra_info=b'**optional free-from "extra info goes here: It is meant for ownership and dating info.**', content=b'--this is where content goes--')

The offset tells us where the content actually starts. Here, we know it starts at the 184th byte.

In [69]:
parts.offset

184

The ``content_hash`` is the sha256 of the original contents, here presented in hex form.

In [70]:
parts.content_hash

'e86755ccc10a29d1b01d6407a490bdfe0f37e9b5cdf8d75b7ac5ea6fac037c38'

In [71]:
parts.extra_info

b'**optional free-from "extra info goes here: It is meant for ownership and dating info.**'

In [72]:
parts.content

b'--this is where content goes--'

## a bit more control on how you get your woven parts 

In [85]:
import json

weaver = HeadWeaver()
content = 'This is some text'
extra_info = {"name": "Thor Whalen", 
              "date": "2013-12-11"}
w = weaver.weave(content.encode(), json.dumps(extra_info).encode())


parts = weaver.unweave_parts(w, interpret_bytes=True, decode_content=True, decode_extra_info=json.loads)
parts

UnWovenBaseHeaderWeave(offset=141, content_hash='2263d8dd95ccfe1ad45d732c6eaaf59b3345e6647331605cb15aae52002dff75', extra_info={'name': 'Thor Whalen', 'date': '2013-12-11'}, content='This is some text')

In [88]:
assert isinstance(parts.extra_info, dict)  # I'm now getting extra_info in the form of a dict
parts.extra_info

{'name': 'Thor Whalen', 'date': '2013-12-11'}

In [90]:
assert isinstance(parts.content, str)  # content is as a str
parts.content

'This is some text'